# Fix for GDrive

In [ ]:
# !pip install -U --no-cache-dir gdown --pre > /dev/null


# Mount GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# Download and Extract Dataset

In [ ]:
# !gdown "1Hkn-xh9gqjWOCkFA0OwzvWPDRsIPwdSv"
# !unzip -o "AAA_DATASET_v7.zip" > /dev/null

# Imports

In [ ]:
import os
import datetime
import matplotlib
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm

matplotlib.use('Agg')
plt.rcParams["font.size"] = 16

# Config

In [ ]:
PROBLEM = "Curvature_2_ECAP"

MODEL_NAME = "MultiViewUNet_Curvature_2_TAWSS_2_ECAP"
DATASET_DIR = "/content/Images/"
TRAIN_DIR = "Train/"
TEST_DIR = "Test/"
INPUT_DIR = PROBLEM.split("_2_")[0]
TARGET_DIR = PROBLEM.split("_2_")[1]
PRED_DIR = "/content/drive/MyDrive/Research/TAVI/Predictions/"
IMG_SIZE = 256
BATCH_SIZE = 16
BUFFER_SIZE = 1000
VAL_SPLIT = 0.2
LEARNING_RATE = 0.001
N_EPOCHS = 300
PATIENCE = 30

MODEL_PATH = ""
SECOND_MODEL_PATH = ""

EXP_NAME = f"{MODEL_NAME}"

# DataLoader

In [ ]:
def load_data_from_dir(path: str) -> tf.data.Dataset:
    return tf.keras.utils.image_dataset_from_directory(
        directory=path,
        labels=None,
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        image_size=(IMG_SIZE, IMG_SIZE),
        shuffle=False,
        seed=42,
        interpolation='bilinear',
        follow_links=False,
        crop_to_aspect_ratio=False
    )

# Load Dataset

In [ ]:
trainX = load_data_from_dir(os.path.join(DATASET_DIR, TRAIN_DIR, INPUT_DIR))
trainY = load_data_from_dir(os.path.join(DATASET_DIR, TRAIN_DIR, TARGET_DIR))
testX = load_data_from_dir(os.path.join(DATASET_DIR, TEST_DIR, INPUT_DIR))
testY = load_data_from_dir(os.path.join(DATASET_DIR, TEST_DIR, TARGET_DIR))

train_ds = tf.data.Dataset.zip((trainX, trainY))
test_ds = tf.data.Dataset.zip((testX, testY))

print(train_ds.element_spec)
print(test_ds.element_spec)

# Normalization

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (
    normalization_layer(x), normalization_layer(y)))
test_ds = test_ds.map(lambda x, y: (
    normalization_layer(x), normalization_layer(y)))

# Optimization

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_batches = (
    train_ds
    .cache()
    .shuffle(BUFFER_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_batches = (
    test_ds
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

# Loss Functions / Metrics

In [ ]:
def attention_mse(y_true, y_pred):
    _y_true = y_true[y_true != 1.0]
    _y_pred = y_pred[y_true != 1.0]
    squared_difference = tf.square(_y_true - _y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


def attention_mae(y_true, y_pred):
    _y_true = y_true[y_true != 1.0]
    _y_pred = y_pred[y_true != 1.0]
    squared_difference = tf.abs(_y_true - _y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)

# Load Models

In [ ]:
model = tf.keras.models.load_model(
    MODEL_PATH,
    custom_objects={"attention_mse": attention_mse}
)
second_model = tf.keras.models.load_model(
    SECOND_MODEL_PATH,
    custom_objects={"attention_mse": attention_mse}
)


# Prediction

In [ ]:
prediction = model.predict(test_ds)
second_prediction = second_model.predict(prediction)

for idx, image in enumerate(second_prediction.unbatch()):
    print(image.shape)
    break